In [1]:
import theano
import theano.tensor as tt
import pymc3 as pm
import numpy as np
import arviz as az
from sklearn.feature_selection import mutual_info_regression

In [2]:
@theano.compile.ops.as_op(itypes=[tt.dscalar, tt.dscalar],
                          otypes=[tt.dscalar])
def numpy_normal(s1,s2):
        #s1,s2 = X
        x = np.random.normal(0, s1, size=10_000)
        y = np.random.normal(0, s2, size=10_000)

        out = x+y
        I = mutual_info_regression(x.reshape((-1,1)),out, discrete_features=False)[0]
        return np.array(I, dtype=np.float64)

In [3]:
with pm.Model() as model:
    s1 = pm.Uniform("s1", 0.1,26)
    
    s2 = pm.Uniform("s2", 0.1,26)
    
    Is = numpy_normal(s1,s2)
    obs = pm.Normal("obs", Is > 4, 0.1, observed = 1)
    
    trace = pm.sample(1000, cores=1, return_inferencedata=False)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Initializing NUTS failed. Falling back to elementwise auto-assignment.
Sequential sampling (2 chains in 1 job)
CompoundStep
>Slice: [s2]
>Slice: [s1]


Sampling 1 chain for 583 tune and 0 draw iterations (583 + 0 draws total) took 429 seconds.
No posterior samples. Unable to run convergence checks
